In [98]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime as dt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, \
cross_validate, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold

## Read data

In [217]:
train = pd.read_csv('./datasets/train_classification.csv')
test = pd.read_csv('./datasets/test_classification.csv')

## 1) Data pre-processing

Put the data pre-processing code. You don't need to explain it. You may use the same code from last quarter.

In [218]:
train['has_missing'] = train.isnull().any(axis=1).astype(int)
test['has_missing'] = test.isnull().any(axis=1).astype(int)

In [219]:
# Define a function to categorize the property types
def categorize_property(property_type):
    if 'Entire' in property_type:
        return 'Entire Home/Apartment'
    elif 'Private' in property_type:
        return 'Private Room'
    elif 'Shared' in property_type:
        return 'Shared Accommodation'
    elif property_type in ['Room in hotel', 'Room in boutique hotel', 'Boat']:
        return 'Specialty Accommodations'
    else:
        return 'Other'

In [220]:
# overall function to clean training and test data
def clean_data(df):
    
    if 'host_is_superhost' in df.columns:
        df.host_is_superhost = df.host_is_superhost.replace({'t': 1, 'f': 0})
        
    # replace missing values of numeric variables wtih the median
    numeric_columns = df.select_dtypes(include=['number']).columns
    df[numeric_columns] = df[numeric_columns].apply(lambda x: x.fillna(x.median()))

    # replace missing values of categorical variables with the mode 
    categorical_columns = df.select_dtypes(include=['object']).columns
    df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
    
    # replace any 0 values to 1 so that it can go through log transformation
    df['beds'] = df['beds'].replace(0, .01)
    df['accommodates'] = df['accommodates'].replace(0, .01)
    df['number_of_reviews'] = df['number_of_reviews'].replace(0, .01)
    df['reviews_per_month'] = df['reviews_per_month'].replace(0, .01)
    df['number_of_reviews_ltm'] = df['number_of_reviews_ltm'].replace(0, .01)
    df['number_of_reviews_l30d'] = df['number_of_reviews_l30d'].replace(0, .01)
    df['host_total_listings_count'] = df['host_total_listings_count'].replace(0, .01)
    df['host_listings_count'] = df['host_listings_count'].replace(0, .01)
    df['calculated_host_listings_count_private_rooms'] = df['calculated_host_listings_count_private_rooms'].replace(0, .01)
    df['calculated_host_listings_count_shared_rooms'] = df['calculated_host_listings_count_shared_rooms'].replace(0, .01)
    df['calculated_host_listings_count_entire_homes'] = df['calculated_host_listings_count_entire_homes'].replace(0, .01)
    
    df['log_beds'] = np.log(df.beds)
    df['log_accommodates'] = np.log(df.accommodates)
    df['log_reviews'] = np.log(df.number_of_reviews)
    df['log_reviews_per_month'] = np.log(df.reviews_per_month)
    df['log_reviews_ltm'] = np.log(df.number_of_reviews_ltm)
    df['log_reviews_l30d'] = np.log(df.number_of_reviews_l30d)
    df['log_host_total_listings_count'] = np.log(df.host_total_listings_count)
    df['log_host_listings_count'] = np.log(df.host_listings_count)
    df['log_host_listings_count_private_rooms'] = np.log(df.calculated_host_listings_count_private_rooms)
    df['log_host_listings_count_shared_rooms'] = np.log(df.calculated_host_listings_count_shared_rooms)
    df['log_host_listings_count_entire_homes'] = np.log(df.calculated_host_listings_count_entire_homes)

    # calculate the number of days since the host became a host
    df['host_since'] = pd.to_datetime(df['host_since'])
    current_date = dt.now()
    df['host_since_days'] = (current_date - df['host_since']).dt.days
    
    # calculate days since first/last review
    df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    df['first_review_days'] = (current_date - df['first_review']).dt.days
    df['last_review_days'] = (current_date - df['last_review']).dt.days
    
    # make response_rate and acceptance_rate into numeric dtype
    df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype('float')
    df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype('float')
    
    # subgroup property_type (similar levels as room_type so discard room predictor)
    df['property_cats'] = df['property_type'].apply(categorize_property)
    
    # extract numeric values from the 'bathrooms' column
    df['bath_numeric'] = df['bathrooms_text'].str.extract('(\d+\.*\d*)', expand=False).astype(float)

    # handle "Half-bath" by assigning a numeric value of 0.5
    df['bath_numeric'] = df.apply(lambda row: 0.5 if 'half' in row['bathrooms_text'].lower() \
                                  else row['bath_numeric'], axis=1)
    
    # to binary
    df.host_identity_verified = df.host_identity_verified.replace({'t': 1, 'f': 0})
    df.host_has_profile_pic = df.host_has_profile_pic.replace({'t': 1, 'f': 0})
    df.has_availability = df.has_availability.replace({'t': 1, 'f': 0})
    df.instant_bookable = df.instant_bookable.replace({'t': 1, 'f': 0})

    # drop the modified/redundant columns
    df.drop(columns = ['host_since', 'first_review', 'last_review', 'property_type', 'bathrooms_text', \
                       'number_of_reviews', 'reviews_per_month', 'number_of_reviews_ltm', \
                       'number_of_reviews_l30d', 'host_total_listings_count', 'host_listings_count', \
                      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', \
                       'calculated_host_listings_count_entire_homes'], inplace = True)
    
    # drop predictors that have low corr with log_price and high corr with others to help remove multi-collinearity
    df.drop(columns = ['first_review_days', 'last_review_days', 'host_acceptance_rate', 'host_response_rate', 
                       'availability_60', 'availability_90', 'minimum_minimum_nights', \
                       'maximum_maximum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', \
                       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'], inplace = True)

In [221]:
clean_data(train)
clean_data(test)

In [222]:
# drop the string categorical predictors 
train = train.drop(columns = ['host_neighbourhood', 'neighbourhood_cleansed', 'host_location'])
test = test.drop(columns = ['host_neighbourhood', 'neighbourhood_cleansed', 'host_location'])

In [223]:
# OHE the remaining categorical variables
host_response_time_dummies = pd.get_dummies(train['host_response_time'], prefix='host_response_time')
train = pd.concat([train, host_response_time_dummies], axis = 1)

host_response_time_dummies = pd.get_dummies(test['host_response_time'], prefix='host_response_time')
test = pd.concat([test, host_response_time_dummies], axis = 1)

In [224]:
host_verifications_dummies = pd.get_dummies(train['host_verifications'], prefix='host_verifications')
train = pd.concat([train, host_verifications_dummies], axis = 1)

host_verifications_dummies = pd.get_dummies(test['host_verifications'], prefix='host_verifications')
test = pd.concat([test, host_verifications_dummies], axis = 1)

In [225]:
room_type_dummies = pd.get_dummies(train['room_type'], prefix='room_type')
train = pd.concat([train, room_type_dummies], axis = 1)

room_type_dummies = pd.get_dummies(test['room_type'], prefix='room_type')
test = pd.concat([test, room_type_dummies], axis = 1)

In [226]:
property_cats_dummies = pd.get_dummies(train['property_cats'], prefix='property_cats')
train = pd.concat([train, property_cats_dummies], axis = 1)

property_cats_dummies = pd.get_dummies(test['property_cats'], prefix='property_cats')
test = pd.concat([test, property_cats_dummies], axis = 1)

In [227]:
train = train.drop(columns = ['host_response_time', 'host_verifications', 'room_type', 'property_cats'])
test = test.drop(columns = ['host_response_time', 'host_verifications', 'room_type', 'property_cats'])

In [228]:
# variable spacing
train.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
test.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [229]:
# set response and predictors for scaling
y_train = train.host_is_superhost
X_train = train.drop(columns = ['id', 'host_is_superhost', 'host_id'])
X_test = test.drop(columns = ['id', 'host_id'])

In [231]:
# poly features
poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

In [232]:
# scale the variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

## 2) Hyperparameter tuning

### How many attempts did it take you to tune the model hyperparameters?

It took me around 5 attempts to tune the model hyperparamters.

### Which tuning method did you use (grid search / Bayes search / etc.)?

I used Randomized Search to tune my model.

### What challenges did you face while tuning the hyperparameters, and what actions did you take to address those challenges?

The main challenge I faced was trying to figure out the right amount of Polynomial Features to use, and I came to the conclusion that any above 2 was unnecessary and too complex. I also had to choose a variable selection method first before running KNN, as unncessary predictors would negatively incluence KNN.

### How many hours did you spend on hyperparameter tuning?

Each run of the hyperparamter tuning took around a few minutes. In total, it took me around 1 hour on hyperparameter tuning in addition to the time it took to implement Lasso. 

### Variable Selection Step: Lasso

**Paste the hyperparameter tuning code below. You must show at least one hyperparameter tuning procedure.**

In [1]:
#Hyperparameter tuning code

In [241]:
# Create DataFrames with polynomial features
X_train_poly_df = pd.DataFrame(X_train_scaled, columns=poly.get_feature_names_out(X_train.columns))
X_test_poly_df = pd.DataFrame(X_test_scaled, columns=poly.get_feature_names_out(X_test.columns))

selected_coeffs = []
r = 0
alphas = np.logspace(-1, 3, 200)
kfold = KFold(n_splits = 10, shuffle = True, random_state = 1)

for alpha in alphas:
    lasso = Lasso(alpha = alpha)
    lasso.fit(X_train_scaled,y_train)
    if ((lasso.coef_ == 0).sum() > r) & (len(selected_coeffs) <= r) :
        selected_coeffs.append(np.where(lasso.coef_!=0)[0])
        r = r + 1

model = KNeighborsClassifier()
grid = {'n_neighbors': np.arange(1, 61, 2), 'weights': ['uniform', 'distance'], 'metric': ['manhattan', 'euclidean', 'minkowski'], \
        'p': np.arange(1, 5)}

results = []
for r in range(1, 51):
    
    gcv = RandomizedSearchCV(model, grid, cv = kfold, n_iter = 300, random_state = 10,
                         scoring = 'neg_root_mean_squared_error', n_jobs = -1)
    selected_Xs = X_train_poly_df.iloc[:,selected_coeffs[r-1]]
    selected_predictors = selected_Xs.columns.tolist()
    
    if selected_Xs.shape[1] > 0:
        gcv.fit(selected_Xs, y_train)
    
        cv_rmse = np.sqrt(-gcv.best_score_)
    
        results.append({'r': r,
                    'selected_predictors': selected_predictors,
                    'best_params': gcv.best_params_,
                    'cv_rmse': cv_rmse})
        
optimal_r = min(results, key = lambda x: x['cv_rmse'])

In [242]:
# Extract selected predictors column names
selected_predictors = optimal_r['selected_predictors'][0].split()  # Convert the string to a list of column names

# Find indices of selected predictor column names
selected_predictor_indices = [X_train_poly_df.columns.get_loc(col) for col in selected_predictors]

# Extract selected predictor column names
selected_predictor_names = X_train_poly_df.columns[selected_predictor_indices]

**Paste the optimal hyperparameter values below.**

In [250]:
print(optimal_r['best_params'])

{'weights': 'uniform', 'p': 3, 'n_neighbors': 39, 'metric': 'euclidean'}


## 3) Model

Using the optimal model hyperparameters, train the model, and paste the code below.

In [244]:
# Create and fit the model
model = KNeighborsClassifier(**optimal_r['best_params'])
model.fit(X_train_poly_df[selected_predictor_names], y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=39, p=3)

## 4) Put any ad-hoc steps for further improving model accuracy
For example, scaling up or scaling down the predictions, capping predictions, etc.

Put code below.

#### Implementing `host id`

In [245]:
# Make predictions
pred = model.predict(X_test_poly_df[selected_predictor_names])

In [246]:
id = test.id.values
predicted = pred
submission = pd.DataFrame({'id': id, 'predicted': predicted})
submission = submission.reset_index(drop=True)

In [247]:
# Add 'host_id' to submission
submission['host_id'] = test['host_id']

# Use apply to replace 'predicted' based on 'host_id'
submission['predicted'] = submission.apply(lambda row: train[train['host_id'] == row['host_id']]['host_is_superhost'].values[0] 
                                           if not train[train['host_id'] == row['host_id']].empty else row['predicted'], axis=1)

# Drop 'host_id' from submission
submission = submission.drop(columns=['host_id'])

## 5) Export the predictions in the format required to submit on Kaggle
Put code below.

In [248]:
submission.to_csv('classification_submission.csv', index=False)